<h1><strong> 모델링 및 파이프라인 구축 </strong></h1>
<br>

<hr>

<br>
<h2> 1. Introduction </h2>
 <p style="font-size:16px">📖 개요 : 본 노트북은 서울시 대기오염 데이터와 기상 데이터를 활용하여 미래의 초미세먼지(PM2.5) 및 마세먼지(PM10) 농도 등급을 예측하는 머신러닝 모델을 개발하는 전체 과정을 기록한 리포트이다.
 <p style="font-size:16px">🏁 목표 : 데이터 정제, 특성 공학(PolynomialFeatures), 데이터 스케일링(StandardScaler), 그리고 클래스 불균형 문제 해결을 위한 SMOTEENN 샘플링 기법을 포함한 파이프라인을 구축하여, 최종적으로 로지스틱 회귀 기반의 분류 모델을 학습시킨다.</p>

<br>
<hr>
<br>
<h2> 2. Library Import </h2>


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
from imblearn.combine import SMOTEENN

# 노트북 메모리 로드
%run preprocessing.ipynb

<hr>
<br>
<h2> 3. 로지스틱 회귀 모델 생성 </h2>

In [145]:
time_series_cv = TimeSeriesSplit(n_splits=3)

def get_logistic_model():
    model = LogisticRegressionCV(
        Cs=[0.001, 0.01, 0.1, 1, 10],
        cv=time_series_cv,
        penalty='l2',
        solver='saga',
        random_state=42,
        max_iter=10000,
        scoring='f1',
        n_jobs=-1
    )
    return model

<p style="font-size:16px">
   위 함수는 시계열 데이터에 최적화된 로지스틱 회귀 분류 모델을 생성한다. 시계열 교차 검증을 통해서 시간 순서를 유지하는 전제 하에 최적의 하이퍼파라미터를 찾아내고, 규제 강도를 지정한다.
</p><br>

```
scoring='f1'
```

<p style="font-size:16px">   
   정제된 데이터셋에는 미세먼지(PM10)의 농도가 81 이상인 경우가 극소수이다. 미세먼지 농도가 81 이상인 경우는 매우 치명적이기 때문에, FN(False Negative) 케이스를 최소화하도록 문제를 외부의 파이프라인에서 스모틴(SMOTEENN)으로 해소하기 위해 모델에 따로 옵션은 추가하지 않았다. 이 때, Accuracy 점수는 불균형 데이터셋에 무의미하므로 F1 점수를 평가 기준으로 지정했다.
</p><br>

```
penalty='l2'
solver='saga'
```

<p style="font-size:16px">
   2차 다항 특성을 생성해서 사용하기 때문에 L2 규제(Ridge)를 적용하여 모델의 과적합을 방지하였고, 대용량 데이터셋과 다양한 규제를 지원하는 saga 알고리즘을 사용하였다.
</p><br>

```
Cs=[0.001, 0.01, 0.1, 1, 10],
cv=time_series_cv,
```

<p style="font-size:16px">
   규제 값들의 후보는 0.001, 0.01, 0.1, 1, 10으로 지정하고, 시계열 교차 검증을 진행했다.
</p><br>

In [146]:
def get_logistic_model_improved():
    model = LogisticRegressionCV(
        class_weight='balanced',
        Cs=[0.0001, 0.001, 0.001, 0.1, 1],
        cv=time_series_cv,
        penalty='elasticnet',
        l1_ratios=[0.2, 0.5, 0.7],
        solver='saga',
        random_state=42,
        max_iter=20000,
        scoring='f1',
        n_jobs=-1
    )
    return model

<p style="font-size:16px">
   위 함수는 릿지(Ridge) 규제를 사용했음에도 발생하는 과적합을 완화하기 위해 엘라스틱넷(ElasticNet) 규제와 더욱 강한 성능을 갖는 모델을 생성한다. 과적합이 발생하는 결과를 보고 하이퍼파라미터 값을 계속해서 조정한 결과이며, 이를 통해 얻은 결과는 evaluate.ipynb에서 이어볼 수 있다.
</p><br>

```
class_weight='balanced',
scoring='f1'
```

<p style="font-size:16px">   
   데이터 불균형 문제를 해소하기 위해 <code>class_weight</code> 옵션을 balanced으로 설정하여 클래스 개수의 역수로 가중치를 지정했다. 클래스 개수의 역수로 가중치를 지정하여 실행을 한 결과,  재현율(Recall)이 100%로 FN(False Negative) 데이터를 정확하게 예측하였으므로, 스모틴(SMOTEENN) 기법을 사용할 필요가 없었다.
</p><br>

```
penalty='elasticnet',
l1_ratios=[0.2, 0.5, 0.7],
solver='saga',
```

<p style="font-size:16px">
   L2(Ridge) 규제와 L1(Lasso) 규제를 결합한 <mark>ElasticNet</mark> 규제를 사용하여 과적합 방지 효과를 유도했다. L1 규제의 비율도 교차 검증을 통해 탐색하도록 구성했다.
</p><br>

```
Cs=[0.0001, 0.001, 0.001, 0.1, 1],
cv=time_series_cv,
max_iter=20000,
n_jobs=-1
```

<p style="font-size:16px">
   C 후보값들을 더욱 작게 만들어, 규제 강도를 높이면서 과적합을 덜 하도록 제약을 강하게 걸었다. 파라미터 수렴을 위한 반복이 많기 때문에, CPU 멀티 코어를 사용하도록 옵션을 추가하면서 속도를 개선하였다.
</p><br>
<hr>

<h2> 4. 훈련 및 파이프라인 구축 </h2>

In [147]:
df['target_PM2.5'] = (df['PM2.5'].shift(-1) >= 36).astype(int)
df['target_PM10'] = (df['PM10'].shift(-1) >= 81).astype(int)

df_dropped = df.drop(columns=['date', 'station_name', 'PM2.5', 'PM10', 'target_PM2.5', 'target_PM10'])
# PM2.5 예측용
train_input_25, test_input_25, train_target_25, test_target_25 = train_test_split(
    df_dropped,
    df['target_PM2.5'],
    shuffle=False,
    test_size=0.2
)

# PM10 예측용
train_input_10, test_input_10, train_target_10, test_target_10 = train_test_split(
    df_dropped,
    df['target_PM10'],
    shuffle=False,
    test_size=0.2
)

```
df['target_PM2.5'] = (df['PM2.5'].shift(-1) >= 36).astype(int)
df['target_PM10'] = (df['PM10'].shift(-1) >= 81).astype(int)
df_dropped = df.drop(columns=['date', 'station_name', 'PM2.5', 'PM10', 'target_PM2.5', 'target_PM10'])
```

<p style="font-size:16px"> 데이터셋에 다음 날의 미세먼지(target_PM10) 농도와 초미세먼지(target_PM2.5) 농도 컬럼을 생성하고, 다음 날의 실제 농도를
가져와 각각 36, 81 이상이면 1로 저장하여 이진 분류를 위한 타겟 값을 저장한다. 훈련에 사용되지 않을 컬럼들을 제외한 df_dropped 데이터셋을 새로 생성했다.</p><br>

```
shuffle=False,
test_size=0.2
```

<p style="font-size:16px"> 훈련 세트와 테스트 세트를 생성할 때, 시계열 자료이기 때문에 shuffle은 하지 않으며 테스트 세트의 크기는 전체 데이터의 20%로 지정했다. </p>

In [148]:
pm25_pipeline = ImbPipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('smoteenn', SMOTEENN(smote=SMOTE(k_neighbors=3))),
    ('scaler', StandardScaler()),
    ('model', get_logistic_model())
])
pm25_pipeline.fit(train_input_25, train_target_25)

pm10_pipeline = ImbPipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', get_logistic_model_improved())
])
pm10_pipeline.fit(train_input_10, train_target_10)

,steps,"[('poly', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,degree,2
,interaction_only,False
,include_bias,False
,order,'C'
,copy,True
,with_mean,True
,with_std,True


<p style="font-size:16px"> 

```
('poly', PolynomialFeatures(degree=2, include_bias=False)),
('smoteenn', SMOTEENN(smote=SMOTE(k_neighbors=3))),
('scaler', StandardScaler()),
('model', get_logistic_model())
```

미세먼지 예측 파이프라인 단계와 동일하게 초미세먼지 예측을 하게 되면 성능 및 점수가 클래스 불균형으로 인해 현저하게 낮아지게 된다. 따라서 초세먼지 예측 파이프라인에는 데이터 스케일링 이전에 클래스 불균형 처리를 위한 <mark>SMOTEENN</mark> 샘플링 단계를 추가했으며, 2차 다항 특성 생성을 통해 상호작용하는 특성들의 패턴을 고려하고, 데이터 스케일링, 그리고 로지스틱 회귀 모델로 구성된 비교적 간단한 구조로 설계했다.
<br><br>

```
('poly', PolynomialFeatures(degree=2, include_bias=False)),
('scaler', StandardScaler()),
('model', get_logistic_model_improved())
```

미세먼지 예측 파이프라인에는 스모틴(SMOTEENN)을 사용하지 않고, 규제 방식과 강도를 달리한 <code>get_logistic_model_improved()</code>로 바꾸었다.<br><br>

✅ <mark><strong> SMOTEENN (SMOTE + Edited Nearest Neighbors) </strong></mark>
- 소수 클래스 샘플 주변에서 가상 데이터를 합성해 소수 클래스의 개수를 늘려주는 오버샘플링(over-sampling), SMOTE 이후에 노이즈 샘플을 제거해주는 기법이다.
 SMOTE만을 사용하여 미세먼지 예측 파이프라인을 구축했을 때에는 노이즈로 인해 FP(False Positive)가 폭증하게 되어 정밀도(Precision)이 하락했다. 이러한 FP 폭증을 방지하기 위해서, k개의 최근접 이웃 중 다수의 클래스를 삭제하는 방식인 ENN 기술로 언더샘플링(under-sampling)하여 클래스 경계의 모호를 방지했다.

</p><br><hr>

<h2> 5. 결론 </h2>
<p style="font-size:16px">
    - 이진분류를 위한 로지스틱 회귀 분류 모델의 성능을 비교할 수 있도록 <strong>릿지(Ridge)</strong>와 <strong>엘라스틱넷(ElasticNet)</strong> 규제를 적용한 모델들을 생성<br>
    - <strong>시계열 교차 검증</strong>을 이용하여 <strong>하이퍼파라미터</strong> 값을 추적<br>
    - imblearn에서 제공하는 파이프라인을 이용해 다중 특성 공학, 데이터 스케일링, 샘플링, 모델링 단계를 자동화해서 재현성 및 확장성 확보<br>
    - SMOTE + ENN 기법을 이용하여 소수의 클래스를 늘리는 동시에 불필요한 다수의 클래스를 제거<br><br>
    데이터를 훈련까지 완료한 상태이므로, 이 훈련 세트와 테스트 세트를 이용해 본격적으로 Confusion Matrix, F1 score 등 다양한 평가 지표를 활용하여 성능을 분석해볼 예정이다.
    
    -> notebooks/evaluating.ipynb
</p>